# Jupyter Notebook


In [2]:

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

NameError: name 'M' is not defined

In [2]:
spark = sparknlp.start()

#data = spark.createDataFrame([['OpenAIMP is good American company. It focuses on innovation,career growth . Asif is working here']]).toDF('text')
df_pyspark1 = spark.read.csv('patient_notes.csv',header=True,inferSchema=True)
#df_pyspark1.show()
colfilter = df_pyspark1.select('pn_history')
colfilter.show(truncate=False)

#data.show(truncate=False)

Step = 1:
Data Assembler read string column and create an innotation column.It controls cleaning up of source text and handling of special characters and new lines through 'cleanupMode'

document = DocumentAssembler().setInputCol('text').setOutputCol('document').setCleanupMode('shrink')

Step = 2:
Sentence Detector:Splits sentences in meaningful ways.It explode sentences improves parallelism in large text concentration

sentence = SentenceDetector().setInputCols('document').setOutputCol('sentence')

22/05/15 20:10:02 WARN Utils: Your hostname, hp-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.185 instead (on interface wlp13s0)
22/05/15 20:10:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hp/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hp/.ivy2/cache
The jars for the packages stored in: /home/hp/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-427e2cf1-3882-4b05-ba10-6feb157ac1a0;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.

In [8]:
sentence.setExplodeSentences(True)

SentenceDetector_90360cb6dae6

Step = 3:
Tokenizer: Splits words in meaningful representation for later NLP

In [9]:
tokenizer = Tokenizer().setInputCols('sentence').setOutputCol('token')

In [10]:
tokenizer.setExceptions(['e-mail'])

Tokenizer_f9e485caaeea

Step = 4:
Spell checker:SymmetricDetele or NorvigSweeting:Fix token type

In [11]:
checker = NorvigSweetingModel.pretrained().setInputCols(['token']).setOutputCol('checked')

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[ — ]spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
Download done! Loading the resource.


[ \ ]

[ | ]

[OK!]


Embeddings :Bert or WordEmbedding(glove,genism)-Vector Representation of tokens

In [12]:
embeddings = WordEmbeddingsModel.pretrained().setInputCols(['sentence','checked']).setOutputCol('embeddings')

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ — ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]Download done! Loading the resource.
[OK!]


Named Identity Recognition: 
Identify entities in text(Person,Comapany,Location etc)

In [13]:
ner = NerDLModel.pretrained().setInputCols(['sentence', 'checked','embeddings']).setOutputCol('ner')

ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[ | ]ner_dl download started this may take some time.
Approximate size to download 13.6 MB
Download done! Loading the resource.
[ — ]

2022-05-15 20:12:14.464419: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-15 20:12:14.739854: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_nam

[OK!]


Chunk builder,Ner Converter : 
Reads NER input and build chunks based on labelled entities

In [14]:
converter = NerConverter().setInputCols(['sentence','checked','ner']).setOutputCol('chunk')

In [15]:
from pyspark.ml import Pipeline

In [16]:
pipeline = Pipeline().setStages([document, sentence, tokenizer,checker,embeddings , ner , converter])

Pipeline.fit() is also required by protocol,even if stages are already trained.it returns a Pipeline Model object

In [17]:
model = pipeline.fit(colfilter)

IllegalArgumentException: Dataset does not have any 'text' column

In [17]:
result = model.transform(colfilter)

In [18]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|             checked|          embeddings|                 ner|               chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|OpenAIMP is good ...|[{document, 0, 95...|[{document, 0, 33...|[{token, 0, 7, Op...|[{token, 0, 7, Op...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 17, 24, ...|
|OpenAIMP is good ...|[{document, 0, 95...|[{document, 35, 7...|[{token, 35, 36, ...|[{token, 35, 36, ...|[{word_embeddings...|[{named_entity, 3...|                  []|
|OpenAIMP is good ...|[{document, 0, 95...|[{document, 76, 9...|[{token, 76, 79, ...|[{token, 76, 79, ...|[{word_embeddings...|[{named_entity, 7...|[{

Analysing each step of Pipeline

In [19]:
result.select('sentence.result').show(truncate=False)

+------------------------------------------+
|result                                    |
+------------------------------------------+
|[OpenAIMP is good American company.]      |
|[It focuses on innovation,career growth .]|
|[Asif is working here]                    |
+------------------------------------------+



In [20]:
result.select('checked.result').show(truncate=False)

+-----------------------------------------------+
|result                                         |
+-----------------------------------------------+
|[OpenAIMP, is, good, American, company, .]     |
|[It, focuses, on, innovation,career, growth, .]|
|[Asia, is, working, here]                      |
+-----------------------------------------------+



In [21]:
result.select('ner.result').show(truncate=False)

+-----------------------+
|result                 |
+-----------------------+
|[O, O, O, B-MISC, O, O]|
|[O, O, O, O, O, O]     |
|[B-LOC, O, O, O]       |
+-----------------------+



In [22]:
result.select('ner.begin','ner.end').show(truncate=False)

+------------------------+------------------------+
|begin                   |end                     |
+------------------------+------------------------+
|[0, 9, 12, 17, 26, 33]  |[7, 10, 15, 24, 32, 33] |
|[35, 38, 46, 49, 67, 74]|[36, 44, 47, 65, 72, 74]|
|[76, 81, 84, 92]        |[79, 82, 90, 95]        |
+------------------------+------------------------+



In [23]:
result.select('chunk.result','chunk.begin','chunk.end').show(truncate=False)

+----------+-----+----+
|result    |begin|end |
+----------+-----+----+
|[American]|[17] |[24]|
|[]        |[]   |[]  |
|[Asif]    |[76] |[79]|
+----------+-----+----+



7. Visualize results

In [26]:
from sparknlp_display import NerVisualizer
NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'chunk',
    document_col = 'document'
  )